# Playas

This indicator was used in 2023 Base Blueprint.

A minor change was made in 2023 to better align with how we use zero and nodata values in other indicators

Created by Rua Mordecai, last run by Amy Keister on 9/15/2023

In [114]:
import os
import arcpy

In [115]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Playas\working"

In [116]:
# define raster used for cell size, extent, and snapping
SERaster= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [117]:
# define original playa
playaData= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Playas\probable_playas_v5_shapefiles\playas_v5_public.shp"
JV= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Playas\North_American_Joint_Ventures_Albers\North_American_Joint_Ventures_Albers.shp"

In [118]:
indicatorName = "Playas.tif"

## Start analysis

In [119]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutputWorkspace

In [7]:
# convert to playa health data to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SERaster):
    arcpy.conversion.FeatureToRaster(in_features=playaData, field="healthy", out_raster="Healthy_playa.tif", cell_size=SERaster)

In [8]:
# convert to playa cluster data to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SERaster):
    arcpy.conversion.FeatureToRaster(in_features=playaData, field="cluster", out_raster="Cluster_playa.tif", cell_size=SERaster)

In [9]:
# Create raster of just healthy clusters
out_raster = arcpy.sa.Times("Healthy_playa.tif", "Cluster_playa.tif"); out_raster.save("HealthyCluster_playa.tif")

In [10]:
# Combine rasters for indicator categories
out_raster = arcpy.sa.Plus("HealthyCluster_playa.tif", "Healthy_playa.tif"); out_raster.save("indicatorCategories.tif")

In [11]:
# Add 1 so lowest playa category is 1
out_raster = arcpy.sa.Plus("indicatorCategories.tif", 1); out_raster.save("noZero.tif")

## Make zero values

In [12]:
# reclassify
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
    out_raster = arcpy.sa.Reclassify("noZero.tif", "Value", "NODATA 0;1 1;2 2;3 3", "DATA"); out_raster.save("zero.tif")

In [13]:
# make a shapefile of Playa Lakes JV boundary
with arcpy.EnvManager(outputCoordinateSystem=sr):
    # make a shapefile of only the Playa Lakes JV from JV boundaries
    arcpy.analysis.Select(JV, "PLJV.shp", "JV = 'Playa Lakes'")
    # add and calculate field for raster conversion
    arcpy.management.CalculateField("PLJV.shp", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

In [14]:
# convert Playa Lakes JV boundary to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SERaster):
    arcpy.conversion.FeatureToRaster("PLJV.shp", "raster", "PLJVr.tif", SERaster)

In [15]:
# use Playa Lakes JV to help show extent of indicator
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SERaster):
    # use times to remove data outside PLJV
    out_raster = arcpy.sa.Times("zero.tif", "PLJVr.tif"); out_raster.save(indicatorName)

In [16]:
# clip to Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
    out_raster = arcpy.sa.ExtractByMask(indicatorName, SERaster); out_raster.save(indicatorName)

## Finalize indicator

In [17]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 3:
        return '3 = Healthy playa and part of a larger cluster'
    elif value == 2:
        return '2 = Healthy playa'
    elif value == 1:
        return '1 = Other playa'
    elif value == 0:
        return '0 = Not identified as a playa'   
"""

In [18]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(indicatorName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Playas\\working\\Playas.tif'>

In [120]:
# set code block for next step of adding RGB colors
codeblock = """
def Reclass1(Value):
	if Value == 3:
		return 199
	elif Value == 2:
		return 230
	elif Value == 1:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 3:
		return 0
	elif Value == 2:
		return 117
	elif Value == 1:
		return 204
	else:
		return 255
		
def Reclass3(Value):
	if Value == 3:
		return 199
	if Value == 2:
		return 228
	elif Value == 1:
		return 255
	else:
		return 255
		
"""

In [121]:
# calculate Red field
arcpy.management.CalculateField(indicatorName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(indicatorName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(indicatorName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'Playas.tif'>